# FINAL PROJECT SIC - CHATBOT HỖ TRỢ TRA CỨU THÔNG TIN TRƯỜNG ĐH SƯ PHẠM KỸ THUẬT
- Author: Dang Kim Thanh - AI 1

# I. Xử lý dữ liệu

## Import môi trường cơ sở dữ liệu và thư viện

In [1]:
import os
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key

'73bdec39-1f93-47fc-bd2f-f02883d7be83'

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import PyPDF2
import unidecode
import re
import pdfplumber

c:\Users\ADMIN\OneDrive\Documents\Btap_Code\VisualStudioCode\Python\SIC_AI\RAG-langchain\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Data vectorization


#### convert pdf to .txt


In [5]:
def normalize_filename(filename):
    filename = unidecode.unidecode(filename)
    filename = filename.replace(" ", "_")
    return filename

def remove_page_numbers(text):
    # Loại bỏ số trang chỉ là một con số ở cuối dòng
    lines = text.splitlines()
    cleaned_lines = []
    for line in lines:
        # Nếu dòng chỉ có một con số ở cuối, bỏ nó đi
        if re.match(r'^\s*\d+\s*$', line.strip()):
            continue
        cleaned_lines.append(line)
    return "\n".join(cleaned_lines)

def pdf_to_text(pdf_file, txt_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ""
        for page_num, page in enumerate(pdf.pages):
            # Trích xuất văn bản từ trang
            page_text = page.extract_text()
            
            # Trích xuất và xử lý bảng
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    row_text = "\t".join([cell if cell is not None else "" for cell in row])
                    page_text += "\n" + row_text
            
            # Thêm văn bản của trang vào text
            text += page_text + "\n"
    
    # Loại bỏ số trang khỏi văn bản
    cleaned_text = remove_page_numbers(text)

    # Ghi toàn bộ nội dung đã được làm sạch vào file txt
    with open(txt_file, 'w', encoding='utf-8') as txt:
        txt.write(cleaned_text)

def convert_pdfs_in_folder(pdf_path, txt_path):
    os.makedirs(txt_path, exist_ok=True)
    for filename in os.listdir(pdf_path):
        if filename.endswith('.pdf'):
            nomalized_filename = normalize_filename(filename)
            pdf_file = os.path.join(pdf_path, filename)
            txt_file = os.path.join(txt_path, nomalized_filename.replace('.pdf', '.txt'))
            pdf_to_text(pdf_file, txt_file)
            print(f'Đã chuyển đổi {pdf_file} thành {txt_file}')

#### Đổi tên file

In [4]:
folder_path = 'data/pdf_files'  # Đường dẫn tới thư mục chứa các tệp cần đổi tên

# Duyệt qua tất cả các tệp trong thư mục
for filename in os.listdir(folder_path):
    # Chuyển đổi tên tệp sang không dấu
    new_filename = unidecode.unidecode(filename)
    # Thay thế dấu cách bằng dấu gạch dưới
    new_filename = new_filename.replace(' ', '_')
    # Loại bỏ các ký tự đặc biệt, chỉ giữ lại chữ cái, số và dấu gạch dưới
    new_filename = re.sub(r'[^A-Za-z0-9_.]', '', new_filename)
    
    # Đổi tên tệp
    os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
    print(f"Đã đổi tên tệp: {filename} -> {new_filename}")

Đã đổi tên tệp: 25_NganhCongNgheVatLieu.pdf -> 25_NganhCongNgheVatLieu.pdf
Đã đổi tên tệp: CKM_NganhCNKTCoDienTu.pdf -> CKM_NganhCNKTCoDienTu.pdf
Đã đổi tên tệp: Cong_nghe_che_bien_lam_san.pdf -> Cong_nghe_che_bien_lam_san.pdf
Đã đổi tên tệp: Cong_nghe_che_tao_may.pdf -> Cong_nghe_che_tao_may.pdf
Đã đổi tên tệp: Cong_nghe_in.pdf -> Cong_nghe_in.pdf
Đã đổi tên tệp: Cong_nghe_ky_thuat_cong_trinh_xay_dung.pdf -> Cong_nghe_ky_thuat_cong_trinh_xay_dung.pdf
Đã đổi tên tệp: Cong_nghe_ky_thuat_co_khi.pdf -> Cong_nghe_ky_thuat_co_khi.pdf
Đã đổi tên tệp: Cong_nghe_Ky_thuat_Dien_tu_-_Truyen_thong.pdf -> Cong_nghe_Ky_thuat_Dien_tu__Truyen_thong.pdf
Đã đổi tên tệp: Cong_nghe_ky_thuat_dieu_khien_va_tu_dong_hoa.pdf -> Cong_nghe_ky_thuat_dieu_khien_va_tu_dong_hoa.pdf
Đã đổi tên tệp: Cong_nghe_Ky_thuat_Hoa_hoc.pdf -> Cong_nghe_Ky_thuat_Hoa_hoc.pdf
Đã đổi tên tệp: Cong_nghe_ky_thuat_may_tinh.pdf -> Cong_nghe_ky_thuat_may_tinh.pdf
Đã đổi tên tệp: Cong_nghe_ky_thuat_moi_truong.pdf -> Cong_nghe_ky_thuat_mo

In [7]:
pdf_file = 'data/pdf_files/Cong_nghe_che_bien_lam_san.pdf'  # Đường dẫn đến tệp PDF
txt_file = 'data/txt_files/output.txt' # Đường dẫn đến tệp .txt bạn muốn lưu
pdf_to_text(pdf_file, txt_file)

In [8]:
pdf_path = 'data/pdf_files'
txt_path = 'data/txt_files'
convert_pdfs_in_folder(pdf_path, txt_path)

Đã chuyển đổi data/pdf_files\25_NganhCongNgheVatLieu.pdf thành data/txt_files\25_NganhCongNgheVatLieu.txt
Đã chuyển đổi data/pdf_files\CKM_NganhCNKTCoDienTu.pdf thành data/txt_files\CKM_NganhCNKTCoDienTu.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_che_bien_lam_san.pdf thành data/txt_files\Cong_nghe_che_bien_lam_san.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_che_tao_may.pdf thành data/txt_files\Cong_nghe_che_tao_may.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_in.pdf thành data/txt_files\Cong_nghe_in.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_ky_thuat_cong_trinh_xay_dung.pdf thành data/txt_files\Cong_nghe_ky_thuat_cong_trinh_xay_dung.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_ky_thuat_co_khi.pdf thành data/txt_files\Cong_nghe_ky_thuat_co_khi.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_Ky_thuat_Dien_tu__Truyen_thong.pdf thành data/txt_files\Cong_nghe_Ky_thuat_Dien_tu__Truyen_thong.txt
Đã chuyển đổi data/pdf_files\Cong_nghe_ky_thuat_dieu_khien_va_tu_dong_hoa.pdf thành data/txt_files\Cong_nghe

## Xử lý data


#### Khai báo hàm xử lý

In [9]:
def clean_text_line(line):
    # Xóa các khoảng trắng đầu/cuối dòng và các ký tự không cần thiết
    line = line.strip()
    line = re.sub(r'\s+', ' ', line)  # Thay nhiều khoảng trắng bằng một khoảng trắng
    return line

def clean_table_data(lines):
    cleaned_data = []
    for line in lines:
        cleaned_line = clean_text_line(line)
        if cleaned_line:  # Bỏ qua dòng trống
            # Tách dữ liệu theo các dấu tab hoặc khoảng trắng lớn
            cleaned_data.append(cleaned_line.replace('\t', ', ').replace('  ', ', '))
    return cleaned_data

def clean_and_merge_lines(lines):
    merged_lines = []
    buffer = ""

    for line in lines:
        stripped_line = line.strip()
        
        # Nếu buffer đang chứa một đoạn văn, nối nó với dòng tiếp theo nếu cần
        if buffer:
            if stripped_line and not stripped_line[0].isupper():
                buffer += " " + stripped_line
            else:
                merged_lines.append(buffer)
                buffer = stripped_line
        else:
            buffer = stripped_line

    # Thêm dòng cuối cùng còn lại trong buffer nếu có
    if buffer:
        merged_lines.append(buffer)
    
    return merged_lines


#### Hàm lưu file

In [10]:
def save_cleaned_data(cleaned_data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for line in cleaned_data:
            f.write(line + '\n')

def process_txt_file(txt_file, cleaned_txt_file):
    with open(txt_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    merged_lines = clean_and_merge_lines(lines)
    cleaned_data = clean_table_data(merged_lines)
    
    save_cleaned_data(cleaned_data, cleaned_txt_file)

def process_all_files_in_folder(input_folder, output_folder):
    # Tạo thư mục đích nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, filename)
            process_txt_file(input_file, output_file)
            print(f'Đã xử lý {input_file} và lưu vào {output_file}')

In [11]:
input_folder = 'data/txt_files'
output_folder = 'data/cleaned_txt_files'

process_all_files_in_folder(input_folder, output_folder)

Đã xử lý data/txt_files\25_NganhCongNgheVatLieu.txt và lưu vào data/cleaned_txt_files\25_NganhCongNgheVatLieu.txt
Đã xử lý data/txt_files\CKM_NganhCNKTCoDienTu.txt và lưu vào data/cleaned_txt_files\CKM_NganhCNKTCoDienTu.txt
Đã xử lý data/txt_files\Cong_nghe_che_bien_lam_san.txt và lưu vào data/cleaned_txt_files\Cong_nghe_che_bien_lam_san.txt
Đã xử lý data/txt_files\Cong_nghe_che_tao_may.txt và lưu vào data/cleaned_txt_files\Cong_nghe_che_tao_may.txt
Đã xử lý data/txt_files\Cong_nghe_in.txt và lưu vào data/cleaned_txt_files\Cong_nghe_in.txt
Đã xử lý data/txt_files\Cong_nghe_ky_thuat_cong_trinh_xay_dung.txt và lưu vào data/cleaned_txt_files\Cong_nghe_ky_thuat_cong_trinh_xay_dung.txt
Đã xử lý data/txt_files\Cong_nghe_ky_thuat_co_khi.txt và lưu vào data/cleaned_txt_files\Cong_nghe_ky_thuat_co_khi.txt
Đã xử lý data/txt_files\Cong_nghe_Ky_thuat_Dien_tu_-_Truyen_thong.txt và lưu vào data/cleaned_txt_files\Cong_nghe_Ky_thuat_Dien_tu_-_Truyen_thong.txt
Đã xử lý data/txt_files\Cong_nghe_Ky_thuat


## Load data


In [ ]:
class UTF8TextLoader(TextLoader):
    def __init__(self, file_path):
        super().__init__(file_path, encoding='utf-8')

pdf_path = 'data/cleaned_txt_files'
loader = DirectoryLoader(path=pdf_path, glob='*.txt', loader_cls=UTF8TextLoader)
documents = loader.load()

## Data split

In [ ]:
separators = [
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma
    "\u3001",  # Ideographic comma
    "\uff0e",  # Fullwidth full stop
    "\u3002",  # Ideographic full stop
    "",
]
chunk_size = 500
chunk_overlap = 50

def word_count(text):
    return len(text.split())

In [ ]:
#splitter
document_splitter = RecursiveCharacterTextSplitter(
    separators = separators,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = word_count,
    is_separator_regex = False
)
#embedding
#vector_store

In [ ]:
chunks = document_splitter.split_documents(documents)
chunks

## Embedding model

In [ ]:
embedding_model = HuggingFaceEmbeddings()

#### Test model embedding

In [ ]:
embedded_txt = embedding_model.embed_query('Học phần pháp luật đại cương trang bị cho sinh viên điều gì?')
embedded_txt

## Vector store


In [ ]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index('docs-rag-chatbot')

#### Lưu vector data lên cloud

In [ ]:
db = PineconeVectorStore.from_documents(documents=chunks, embedding=embedding_model, index_name="docs-rag-chatbot", 
                                            namespace="docs-ute-store" )